In [1]:
!pip install graphviz
!pip install folium
!pip install unidecode


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import csv
from graphviz import Digraph
import folium
from IPython.display import Image, display
from unidecode import unidecode

In [3]:
# Definimos la ruta del archivo CSV que se va a procesar
archivo_csv = "/work/data.csv"

# Creamos una lista para almacenar los datos procesados desde el archivo CSV
lista_datos = []

# Abrimos el archivo CSV en modo lectura
with open(archivo_csv, mode='r', newline='') as file:
    # Creamos un lector de CSV que interpreta las filas como diccionarios
    csv_reader = csv.DictReader(file)

    # Iteramos a través de cada fila del archivo CSV
    for row in csv_reader:
        # Normalizamos y convertimos a minúsculas ciertos campos
        row['title'] = unidecode(row['title']).lower()
        row['department'] = unidecode(row['department']).lower()
        row['city'] = unidecode(row['city']).lower()
        row['property_type'] = unidecode(row['property_type']).lower()
        row['operation_type'] = unidecode(row['operation_type']).lower()
        # Convertimos campos numéricos a tipo float
        row['latitude'] = float(row['latitude'])
        row['longitude'] = float(row['longitude'])
        row['surface_total'] = float(row['surface_total'])
        row['surface_covered'] = float(row['surface_covered'])
        row['bedrooms'] = float(row['bedrooms'])
        row['bathrooms'] = float(row['bathrooms'])
        row['price'] = float(row['price'])
        # Agregamos la fila procesada a la lista de datos
        lista_datos.append(row)

print(lista_datos)

[{'title': 'apartamento en arriendo/venta en barranquilla villa santos cod. vare83401', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'apartamento', 'latitude': 11.013, 'longitude': -74.836, 'surface_total': 70.0, 'surface_covered': 70.0, 'bedrooms': 1.0, 'bathrooms': 2.0, 'operation_type': 'venta', 'price': 250000000.0}, {'title': 'casa en venta en cali ciudad jardn cod. vvlz3039', 'department': 'valle del cauca', 'city': 'cali', 'property_type': 'casa', 'latitude': 3.368, 'longitude': -76.531, 'surface_total': 420.0, 'surface_covered': 420.0, 'bedrooms': 5.0, 'bathrooms': 5.0, 'operation_type': 'venta', 'price': 950000000.0}, {'title': 'casa en arriendo/venta en barranquilla bellavista cod. vfnc10498', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'casa', 'latitude': 10.997, 'longitude': -74.791, 'surface_total': 83.0, 'surface_covered': 150.0, 'bedrooms': 3.0, 'bathrooms': 3.0, 'operation_type': 'venta', 'price': 320000000.0}, {'title': 'c

In [4]:
class NodoArbol:
    def __init__(self, datos):
        self.metricas = [datos["price"] / datos["surface_total"], datos["surface_covered"] / datos["surface_total"]]
        self.datos = datos
        self.izquierda = None
        self.derecha = None
        self.altura = 1

        
    def mostrar_ubicacion_en_mapa(self):
        # Extrae la latitud y longitud de los datos proporcionados (supongo que son un diccionario).
        latitud = self.datos['latitude']
        longitud = self.datos['longitude']
        # Crea un mapa en Folium centrado en la ubicación dada (latitud y longitud) con un nivel de zoom de 15.
        mapa = folium.Map(location=[latitud, longitud], zoom_start=15)
        # Agrega un marcador en el mapa en la ubicación dada.
        folium.Marker([latitud, longitud]).add_to(mapa)
        # Muestra el mapa en el entorno actual (esto asume que estás ejecutando este código en un entorno que puede mostrar mapas, como un Jupyter Notebook).
        display(mapa)

class ArbolAVL:
    def __init__(self):
        self.raiz = None

    def altura(self, nodo):
        if not nodo:
            return 0
        return nodo.altura

    def balance(self, nodo):
        if not nodo:
            return 0
        return self.altura(nodo.izquierda) - self.altura(nodo.derecha)

    def rotar_izquierda(self, nodo):
        nueva_raiz = nodo.derecha
        nodo.derecha = nueva_raiz.izquierda
        nueva_raiz.izquierda = nodo
        nodo.altura = max(self.altura(nodo.izquierda), self.altura(nodo.derecha)) + 1
        nueva_raiz.altura = max(self.altura(nueva_raiz.izquierda), self.altura(nueva_raiz.derecha)) + 1
        return nueva_raiz

    def rotar_derecha(self, nodo):
        nueva_raiz = nodo.izquierda
        nodo.izquierda = nueva_raiz.derecha
        nueva_raiz.derecha = nodo
        nodo.altura = max(self.altura(nodo.izquierda), self.altura(nodo.derecha)) + 1
        nueva_raiz.altura = max(self.altura(nueva_raiz.izquierda), self.altura(nueva_raiz.derecha)) + 1
        return nueva_raiz
    
    def nodo_minimo(self, nodo):
        current = nodo
        while current.izquierda:
            current = current.izquierda
        return current

    def insertar(self, raiz, datos):
        if not raiz:
            return NodoArbol(datos)

        metricas_nuevo_nodo = [datos["price"] / datos["surface_total"], datos["surface_covered"] / datos["surface_total"]]
        metricas_raiz = raiz.metricas

        if metricas_nuevo_nodo < metricas_raiz:
            raiz.izquierda = self.insertar(raiz.izquierda, datos)
        elif metricas_nuevo_nodo > metricas_raiz:
            raiz.derecha = self.insertar(raiz.derecha, datos)
        else:
            if metricas_nuevo_nodo[1] < metricas_raiz[1]:
                raiz.izquierda = self.insertar(raiz.izquierda, datos)
            else:
                raiz.derecha = self.insertar(raiz.derecha, datos)

        raiz.altura = max(self.altura(raiz.izquierda), self.altura(raiz.derecha)) + 1
        balance = self.balance(raiz)

        if balance > 1:
            if metricas_nuevo_nodo < raiz.izquierda.metricas:
                return self.rotar_derecha(raiz)
            else:
                raiz.izquierda = self.rotar_izquierda(raiz.izquierda)
                return self.rotar_derecha(raiz)
        if balance < -1:
            if metricas_nuevo_nodo > raiz.derecha.metricas:
                return self.rotar_izquierda(raiz)
            else:
                raiz.derecha = self.rotar_derecha(raiz.derecha)
                return self.rotar_izquierda(raiz)

        return raiz

    def insert(self, datos):
        self.raiz = self.insertar(self.raiz, datos)

    def generar_dot(self, filename):
        # Crea un objeto Digraph de Graphviz en formato SVG.
        dot = Digraph(format="svg")

        # Define una función interna llamada agregar_nodo para agregar nodos y conexiones al gráfico.
        def agregar_nodo(nodo):
            if nodo:
                # Agrega un nodo al gráfico con su ID y datos del nodo.
                dot.node(str(id(nodo)), str(nodo.datos))
                 # Si hay un nodo a la izquierda, agrega una conexión etiquetada como "izq".
                if nodo.izquierda:
                    dot.edge(str(id(nodo)), str(id(nodo.izquierda)), label="izq")
                    # Llama recursivamente a agregar_nodo para procesar los nodos a la izquierda.
                    agregar_nodo(nodo.izquierda)
                if nodo.derecha:
                    dot.edge(str(id(nodo)), str(id(nodo.derecha)), label="der")
                    # Llama recursivamente a agregar_nodo para procesar los nodos a la derecha.
                    agregar_nodo(nodo.derecha)
        # Verifica si el árbol tiene una raíz antes de comenzar a generar el gráfico.
        if self.raiz:
            # Llama a agregar_nodo con la raíz del árbol para comenzar el proceso de generación.
            agregar_nodo(self.raiz)
        # Genera el gráfico en el formato especificado (SVG) y lo guarda en el archivo especificado por 'filename'.    
        dot.render(filename)

1) Insertar un nodo (IMAGEN PESADA: para verla descargar el svg en la carpeta insertar).

In [5]:
arbol = ArbolAVL()
for datos in lista_datos:
    arbol.insert(datos)
arbol.generar_dot("insertar/imagen_arbol")

3) Buscar un nodo utilizando la métrica dada.

In [6]:
def buscar_nodo_por_metricas(self, raiz, metricas_busqueda):
    if not raiz:
        return None
    if metricas_busqueda == raiz.metricas:
        return raiz
    elif metricas_busqueda < raiz.metricas:
        return self.buscar_nodo_por_metricas(raiz.izquierda, metricas_busqueda)
    else:
        return self.buscar_nodo_por_metricas(raiz.derecha, metricas_busqueda)
ArbolAVL.buscar_nodo_por_metricas = buscar_nodo_por_metricas

In [7]:
datos_busqueda = { 'surface_total': 202.0, 'surface_covered': 202.0, 'price': 1300000000.0 }
#datos_busqueda = {'surface_total': 1457.0, 'surface_covered': 380.0, 'price': 1730000000}
nodo_encontrado = arbol.buscar_nodo_por_metricas(arbol.raiz, [datos_busqueda["price"] / datos_busqueda["surface_total"], datos_busqueda["surface_covered"] / datos_busqueda["surface_total"]])

if nodo_encontrado:
    print("Nodo encontrado:", nodo_encontrado.datos)
    nodo_encontrado.mostrar_ubicacion_en_mapa()
else:
    print("Nodo no encontrado")

Nodo encontrado: {'title': 'apartamento en venta en bogota santa barbara cod. vgig1144050806', 'department': 'cundinamarca', 'city': 'bogota d.c', 'property_type': 'apartamento', 'latitude': 4.702, 'longitude': -74.039, 'surface_total': 202.0, 'surface_covered': 202.0, 'bedrooms': 3.0, 'bathrooms': 2.0, 'operation_type': 'venta', 'price': 1300000000.0}


.

In [8]:
def eliminar_nodo_por_metricas(self, raiz, metricas_eliminar):
        # Caso base: si el árbol está vacío, simplemente retorna None
        if not raiz:
            return raiz

        # Realiza la búsqueda del nodo a eliminar
        if metricas_eliminar < raiz.metricas:
            raiz.izquierda = self.eliminar_nodo_por_metricas(raiz.izquierda, metricas_eliminar)
        elif metricas_eliminar > raiz.metricas:
            raiz.derecha = self.eliminar_nodo_por_metricas(raiz.derecha, metricas_eliminar)
        else:
            # Si encontramos el nodo a eliminar
            if raiz.izquierda is None:
                temp = raiz.derecha
                raiz = None
                return temp
            elif raiz.derecha is None:
                temp = raiz.izquierda
                raiz = None
                return temp

            # Nodo con dos hijos: obtener el sucesor inorden (el nodo más pequeño en el subárbol derecho)
            temp = self.nodo_min_valor(raiz.derecha)

            # Copiar el contenido del sucesor inorden a este nodo
            raiz.metricas = temp.metricas
            raiz.datos = temp.datos

            # Eliminar el sucesor inorden
            raiz.derecha = self.eliminar_nodo_por_metricas(raiz.derecha, temp.metricas)

        # Actualizar la altura del nodo actual
        raiz.altura = 1 + max(self.altura(raiz.izquierda), self.altura(raiz.derecha))

        # Balancear el árbol
        balance = self.balance(raiz)

        # Casos de rotación
        if balance > 1:
            if metricas_eliminar < raiz.izquierda.metricas:
                return self.rotar_derecha(raiz)
            else:
                raiz.izquierda = self.rotar_izquierda(raiz.izquierda)
                return self.rotar_derecha(raiz)
        if balance < -1:
            if metricas_eliminar > raiz.derecha.metricas:
                return self.rotar_izquierda(raiz)
            else:
                raiz.derecha = self.rotar_derecha(raiz.derecha)
                return self.rotar_izquierda(raiz)

        return raiz

def nodo_min_valor(self, nodo):
        current = nodo
        while current.izquierda is not None:
            current = current.izquierda
        return current
# Agrega el método a la clase ArbolAVL
ArbolAVL.eliminar_nodo_por_metricas = eliminar_nodo_por_metricas
ArbolAVL.nodo_min_valor = nodo_min_valor

In [9]:
#datos_busqueda = { 'surface_total': 202.0, 'surface_covered': 202.0, 'price': 1300000000.0 }
datos_busqueda = {'surface_total': 1457.0, 'surface_covered': 380.0, 'price': 1730000000}

arbol.eliminar_nodo_por_metricas(arbol.raiz,[datos_busqueda["price"] / datos_busqueda["surface_total"], datos_busqueda["surface_covered"] / datos_busqueda["surface_total"]])

4. Buscar todos los nodos que cumplan con una serie de criterios (máximo 3).

In [44]:
def buscar_criterios(self, raiz, datos):
    criterios = []

    # Solicitar al usuario hasta tres criterios de búsqueda
    for i in range(3):
        print(f"Ingrese el criterio de búsqueda #{i + 1} (operation_type, property_type, city, department, bathrooms, price, bedrooms) o deje en blanco para finalizar: ")
        criterio = input()

        # Verificar si el usuario dejó el criterio en blanco para finalizar la selección
        if not criterio.strip():
            break

        if criterio not in ["operation_type", "property_type", "city", "department", "bathrooms", "price", "bedrooms"]:
            print("Criterio no válido. Por favor, ingrese operation_type, property_type, city, department, bathrooms, price o bedrooms.")
            continue

        valor = input(f"Ingrese el valor para el criterio {criterio}: ")

        # Solicitar al usuario la condición para campos numéricos
        if criterio in ["bathrooms", "price", "bedrooms"]:
            condicion = input(f"Elija la condición para el campo {criterio} (<, <=, =, >=, >): ")
            if condicion not in ["<", "<=", "=", ">=", ">"]:
                print("Condición no válida. Por favor, elija <, <=, =, >= o >.")
                continue
            valor = float(valor)  # Convertir el valor a número si es necesario

            # Agregar el criterio con la condición y el valor a la lista de criterios
            criterios.append((criterio, condicion, valor))
        else:
            # Agregar el criterio sin condición a la lista de criterios
            criterios.append((criterio, None, valor))

    if not criterios:
        print("No se ingresaron criterios de búsqueda.")
        return

    print("Los resultados de su búsqueda son los siguientes:")
    print("")

    coincidencias = []

    # Función interna para realizar la búsqueda en el árbol AVL
    def busqueda(raiz, datos):
        nonlocal coincidencias
        if raiz is not None:
            # Verificar si el nodo actual cumple con todos los criterios de búsqueda
            cumple_criterios = all(
                (condicion is None and
                 (criterio in ["department", "operation_type", "property_type", "city"] and raiz.datos[criterio] == valor) or
                 (criterio not in ["department", "operation_type", "property_type", "city"] and (
                     (condicion == "<" and raiz.datos[criterio] < valor) or
                     (condicion == "<=" and raiz.datos[criterio] <= valor) or
                     (condicion == "=" and raiz.datos[criterio] == valor) or
                     (condicion == ">=" and raiz.datos[criterio] >= valor) or
                     (condicion == ">" and raiz.datos[criterio] > valor)
                 )))
                for criterio, condicion, valor in criterios
            )

            # Si el nodo cumple con los criterios, agregarlo a la lista de coincidencias
            if cumple_criterios:
                coincidencias.append(raiz)

            # Realizar la búsqueda en los nodos hijos
            if raiz.izquierda is not None:
                busqueda(raiz.izquierda, raiz.izquierda.datos)
            if raiz.derecha is not None:
                busqueda(raiz.derecha, raiz.derecha.datos)

    # Llamar a la función de búsqueda con la raíz del árbol y los datos
    busqueda(raiz, datos)

    # Imprimir los resultados de la búsqueda
    for k in coincidencias:
        print(k.datos)
        k.mostrar_ubicacion_en_mapa()

# Asignar la función buscar_criterios al objeto arbol de tipo ArbolAVL
ArbolAVL.buscar_criterios = buscar_criterios

# Llamar a la función buscar_criterios en el objeto arbol
arbol.buscar_criterios(arbol.raiz, arbol.raiz.datos)

Ingrese el criterio de búsqueda #1 (operation_type, property_type, city, department, bathrooms, price, bedrooms) o deje en blanco para finalizar: 
Ingrese el criterio de búsqueda #2 (operation_type, property_type, city, department, bathrooms, price, bedrooms) o deje en blanco para finalizar: 
Criterio no válido. Por favor, ingrese operation_type, property_type, city, department, bathrooms, price o bedrooms.
Ingrese el criterio de búsqueda #3 (operation_type, property_type, city, department, bathrooms, price, bedrooms) o deje en blanco para finalizar: 
Los resultados de su búsqueda son los siguientes:

{'title': 'apartamento en arriendo en barranquilla villa santos cod. aadc-41574', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'apartamento', 'latitude': 11.017, 'longitude': -74.837, 'surface_total': 78.0, 'surface_covered': 78.0, 'bedrooms': 2.0, 'bathrooms': 2.0, 'operation_type': 'arriendo', 'price': 1200000.0}


{'title': 'apartamento en arriendo en barranquilla ciudad jardn cod. aadc41940', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'apartamento', 'latitude': 10.992, 'longitude': -74.818, 'surface_total': 94.0, 'surface_covered': 94.0, 'bedrooms': 2.0, 'bathrooms': 3.0, 'operation_type': 'arriendo', 'price': 1700000.0}


{'title': 'apartamento en arriendo en barranquilla betania cod. aadn758180', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'apartamento', 'latitude': 10.986, 'longitude': -74.812, 'surface_total': 62.0, 'surface_covered': 62.0, 'bedrooms': 2.0, 'bathrooms': 2.0, 'operation_type': 'arriendo', 'price': 1200000.0}


5. Realizar el recorrido por niveles del arbol

In [11]:
#Recorrido por niveles del arbol

def levels_nr_recursive(node, level=0):
    if node is None:
        return

    if level == 0:
        print(node.datos)

    if node.izquierda:
        print(node.izquierda.datos)

    if node.derecha:
        print(node.derecha.datos)

    levels_nr_recursive(node.izquierda, level + 1)
    levels_nr_recursive(node.derecha, level + 1)

# Ejemplo de uso:
# Supongamos que tienes un árbol con una raíz llamada 'arbol.raiz'

# Llamada a la función para empezar el recorrido por niveles desde la raíz:
levels_nr_recursive(arbol.raiz)

{'title': 'lote en venta en barranquilla olaya herrera cod. vbgg-100693', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'lote', 'latitude': 10.988, 'longitude': -74.807, 'surface_total': 380.0, 'surface_covered': 380.0, 'bedrooms': 3.0, 'bathrooms': 1.0, 'operation_type': 'venta', 'price': 500000000.0}
{'title': 'casa en arriendo/venta en barranquilla caujaral cod. arei-16605', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'casa', 'latitude': 11.005, 'longitude': -74.815, 'surface_total': 1326.0, 'surface_covered': 1050.0, 'bedrooms': 4.0, 'bathrooms': 6.0, 'operation_type': 'arriendo', 'price': 25000000.0}
{'title': 'apartamento en arriendo/venta en barranquilla villa santos cod. vare83401', 'department': 'atlantico', 'city': 'barranquilla', 'property_type': 'apartamento', 'latitude': 11.013, 'longitude': -74.836, 'surface_total': 70.0, 'surface_covered': 70.0, 'bedrooms': 1.0, 'bathrooms': 2.0, 'operation_type': 'venta', 'price': 250000000.

a. Hallar el nivel de el nodo_encontrado

In [13]:
def guardar(root, target_node, nivel):
    level = 0
    def buscar(node, target_node, lvl) -> None:
        nonlocal level # Permite modificar la variable level desde el ámbito exterior.
        if node is not None:
            if node == target_node:
                level = lvl # Actualiza el nivel cuando se encuentra el nodo deseado.
            else:
                # Realiza una búsqueda recursiva en los nodos izquierdo y derecho del nodo actual.
                buscar(node.izquierda, target_node, lvl + 1)
                buscar(node.derecha, target_node, lvl + 1)
        return level
    # Llama a la función buscar para encontrar el nivel del nodo target_node en el árbol.
    level = buscar(root,target_node,0)
    return level
    
# Agrega la función guardar como un método a la clase ArbolAVL.
ArbolAVL.guardar = guardar
print("El nivel del nodo encontrado es: ",guardar(arbol.raiz, nodo_encontrado, 0))

El nivel del nodo encontrado es:  6


b. hallar el grado de balanceo de un nodo

In [16]:
arbol.balance(nodo_encontrado)

-1

c. Buscar el padre del nodo_encontrado

In [12]:
#Buscar el padre del nodo_encontrado
def find_father(root,son):
    parent = None
    targetlvl = None
    #Creamos una función la cual nos permitira encontrar el nivel del padre
    def look_father_level(node, target_node, level):
        nonlocal targetlvl
        if node is not None:
            if node == target_node: #Comprobamos si el nodo del arbol recorrido es el hijo
                targetlvl = level - 1 #Al nivel del hijo le restamos 1 para obtener el nivel de padre
            else:
                #Iteramos hasta encontrar el nodo hijo aumentando el nivel en 1
                look_father_level(node.izquierda, target_node, level + 1)
                look_father_level(node.derecha, target_node, level + 1)
        return targetlvl
    #Creamos una funcion que se encargara de obtener el padre
    def look_parent(node, targetlvl, target_node, level):
        nonlocal parent
        if node is not None:
            if level == targetlvl: #Comprobamos si el nivel actual es el nivel del padre (targetlvl)
                if node.izquierda != None:
                    if node.izquierda == target_node or node.derecha == target_node: #Si el hijo de la izquierda del nodo actual es igual al hijo ingresado con anterioridad (target_node) habremos encontrado el padre
                        parent = node
                else:
                    if node.derecha != None:
                        if node.derecha == target_node or node.izquierda == target_node:#Si el hijo de la derecha del nodo actual es igual al hijo ingresado con anterioridad (target_node) habremos encontrado el padre
                            parent = node
            else:
                #Iteramos hasta encontrar el padre aumentando el nivel del nodo en 1
                look_parent(node.izquierda, targetlvl, target_node, level + 1)
                look_parent(node.derecha, targetlvl, target_node, level + 1)
        return parent

    fatherlvl = look_father_level(root,son,0)
    father = look_parent(root,targetlvl, son, 0)
    return father

ArbolAVL.find_father = find_father

father= find_father(arbol.raiz, nodo_encontrado)
print("El nodo padre es: ",father.datos)
father.mostrar_ubicacion_en_mapa()

El nodo padre es:  {'title': 'casa en venta en bogota sotileza cod. vrei18144', 'department': 'cundinamarca', 'city': 'bogota d.c', 'property_type': 'casa', 'latitude': 4.72, 'longitude': -74.077, 'surface_total': 240.0, 'surface_covered': 240.0, 'bedrooms': 4.0, 'bathrooms': 3.0, 'operation_type': 'venta', 'price': 1600000000.0}


d. Encontrar el abuelo del nodo.

In [14]:
# utilizaremos una primera funcion que encontrara el nodo indicado y a su padre y los retornara como tupla
from typing import Optional, Tuple, List
def search(self, metricas = List[float]) -> Tuple[Optional["Node"], Optional["Node"]]:
        elem = metricas
        if not self.raiz:
            return None
        # al empezar asignaremos el nodo a p, y asignaremos su padre como None, en caso de que el nodo encontrado sea la raiz 
        p, pad = self.raiz, None
        while p is not None:
            #si encontramos coincidencia en las metricas retornaremos al nodo encontrado y a su padre
            if elem == p.metricas:
                return p, pad
            else:
                #cada vez que no lo encontremos el nodo actual pasara a ser el padre.
                #pasaremos al hijo izquierdo o derecho del nodo dependiendo de si las metricas que buscamos son menores o mayores a las actuales
                pad = p
                if elem < p.metricas:
                    p = p.izquierda
                else:
                    p = p.derecha
        return p, pad

ArbolAVL.search = search
#la funcion findGrandphater se valdrá de a funcion search para encontrar al abuelo del nodo
def findGrandfather(self, nodo)-> "Node" :
    #la primera vez encontraremos al padre del nodo usando la funcion search
        son, phater = self.search(nodo.metricas)
        #print(son.datos)
        #print(phater.datos)
    #luego al enviar al padre encontrado como parametro a la funcion search encontraremos al padre de este (es decir el abuelo de nuestro nodo)
        phater, grandphater = self.search(phater.metricas)
        return grandphater

ArbolAVL.findGrandfather = findGrandfather

cucho = arbol.findGrandfather(nodo_encontrado)
print(cucho.datos)
cucho.mostrar_ubicacion_en_mapa()

{'title': 'apartamento en venta en bogota la carolina-usaqun cod. vbks2721', 'department': 'cundinamarca', 'city': 'bogota d.c', 'property_type': 'apartamento', 'latitude': 4.707, 'longitude': -74.041, 'surface_total': 198.0, 'surface_covered': 198.0, 'bedrooms': 4.0, 'bathrooms': 4.0, 'operation_type': 'venta', 'price': 1600000000.0}


e. Hallar el tio del nodo_encontrado

In [15]:
#Hallar el tio del nodo_encontrado
def look_uncle(root, target_node, father, grandphater):
    uncle = None
    inicial_node = root
    target_level = None
    
    #Realizamos una funcion que buscara el nivel de nuestro nodo
    def look_node(node, target_node, level):
        nonlocal target_level
        if node is not None:
            if node == target_node:
                target_level = level
            else:
                look_node(node.izquierda, target_node, level + 1)
                look_node(node.derecha, target_node, level + 1)
        return target_level
    
    def lookuncle(node, target_level, level, father, grandphater):
        nonlocal uncle
        if node is not None:
            if level == target_level:
                if cucho.izquierda is not None:
                    if cucho.izquierda == father:
                        uncle = cucho.derecha                   
                else:
                    if cucho.derecha is not None:
                        if cucho.derecha == father:
                            uncle = cucho.izquierda
            else:
                if node.izquierda is not None:
                    lookuncle(node.izquierda, target_level, level + 1, father, grandphater)
                if node.derecha is not None:
                    lookuncle(node.derecha, target_level, level + 1, father, grandphater)
        return uncle

    uncles_level = look_node(root, target_node, 0)
    lookuncle(inicial_node, uncles_level - 1, 0, father, grandphater)
    return uncle

ArbolAVL.look_uncle = look_uncle
print("El tio del nodo ", nodo_encontrado," es: ")
tio = look_uncle(arbol.raiz, nodo_encontrado, father, cucho)
print(tio.datos)
tio.mostrar_ubicacion_en_mapa()


El tio del nodo  <__main__.NodoArbol object at 0x7fcbc5284ca0>  es: 
{'title': 'apartamento en arriendo/venta en bogota las nieves cod. v3412177', 'department': 'cundinamarca', 'city': 'bogota d.c', 'property_type': 'apartamento', 'latitude': 4.611, 'longitude': -74.073, 'surface_total': 42.0, 'surface_covered': 42.0, 'bedrooms': 1.0, 'bathrooms': 1.0, 'operation_type': 'venta', 'price': 350000000.0}


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0e3da084-19e5-4f23-830a-0895f79bda5b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>